# Ranking

In [2]:
%matplotlib inline
from time import time
import numpy as np
import pandas as pd

## Метрики

In [78]:
def dcg(relevances, k=10):
    #Discounted cumulative gain at k (DCG)
    relevances = np.asarray(relevances)[:k]
    n_relevances = len(relevances)
    gain = 2**relevances - 1
    # в формуле discounts + 2 т.к. нумерация начинается с 0
    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(gain / discounts)
 
 
def ndcg(relevances, k=10):
    #Normalized discounted cumulative gain (NDGC)
    best_dcg = dcg(sorted(relevances, reverse=True), k)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, k) / best_dcg

In [83]:
def mean_ndcg(y_true, y_pred, query_ids, k=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, k=k))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, k=k))
    return np.mean(ndcg_scores)

## Загрузка данных

Используемые бинарные данные

In [47]:
query = pd.read_csv("D:/Downloads/Ranking/data/trainDataBinary.csv")

In [48]:
query.head()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,11909,0.048537,0,0.054362,0.000000,0.000000,0.000000,0,0.208262,0.089286,1.000000
1,0,11909,0.000000,0,0.000000,0.000000,0.000000,0.080000,0,0.000000,0.000000,0.000000
2,0,11909,0.014989,0,0.005346,1.000000,1.000000,1.000000,0,1.000000,1.000000,0.166667
3,1,11909,0.048180,0,0.016753,0.000000,1.000000,0.253333,0,0.040667,0.017857,0.000000
4,1,11909,0.254818,0,0.135242,0.615723,0.333333,0.253333,0,0.004727,0.017857,0.527778


In [49]:
X_train = query.iloc[:, 2:] 
y_train = query.loc[:,"relevance"].values 
qid_train = query.loc[:,"qid"].values 
qid_train

array([11909, 11909, 11909, ..., 18218, 18218, 18218], dtype=int64)

In [50]:
query.loc[query.qid == 11909,]

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,11909,0.048537,0,0.054362,0.000000,0.000000,0.000000,0,0.208262,0.089286,1.000000
1,0,11909,0.000000,0,0.000000,0.000000,0.000000,0.080000,0,0.000000,0.000000,0.000000
2,0,11909,0.014989,0,0.005346,1.000000,1.000000,1.000000,0,1.000000,1.000000,0.166667
3,1,11909,0.048180,0,0.016753,0.000000,1.000000,0.253333,0,0.040667,0.017857,0.000000
4,1,11909,0.254818,0,0.135242,0.615723,0.333333,0.253333,0,0.004727,0.017857,0.527778
5,0,11909,0.009279,0,0.007146,0.500362,0.333333,0.466667,0,0.187273,0.303571,0.833333
6,0,11909,0.062099,0,0.067171,0.463787,0.666667,0.413333,0,0.377460,0.035714,0.000000
7,1,11909,1.000000,0,1.000000,0.000000,0.666667,0.480000,0,0.020984,0.017857,0.000000


Перейдем к предсказаниям. Сначала -- обычное предсказание pointwise

Рассмотрим алгоритм RankBoost

In [1]:
from learning2rank.rank import RankNet

ModuleNotFoundError: No module named 'rank'

In [6]:
model = RankBoost(100)
model.fit(X_train, y_train)

NameError: name 'RankBoost' is not defined

In [125]:
y_pred2 = model.predict(X_train)
y_pred2
mean_ndcg(y_true = y_train, y_pred = y_pred, query_ids = qid_train, k=5)

0.8887622753431376